In [8]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import openpyxl

def process_excel_file(file_path):
    """
    Verarbeitet eine Excel-Datei mit Header in Zeile 11, 
    Variablennamen in Zeile 12 und Daten in Zeile 13
    """
    try:
        # Lese die relevanten Zeilen (Python ist 0-basiert, Excel 1-basiert)
        # Daher Zeile 11 = Index 10 etc.
        header_row = pd.read_excel(file_path, header=None, nrows=1, skiprows=10)
        variable_row = pd.read_excel(file_path, header=None, nrows=1, skiprows=11)
        data_row = pd.read_excel(file_path, header=None, nrows=1, skiprows=12)
        
        headers = header_row.values[0]
        variables = variable_row.values[0]
        data = data_row.values[0]
        
        variable_mapping = {}
        for var, header in zip(variables, headers):
            if pd.notna(var) and str(var).strip():
                variable_mapping[str(var).strip()] = str(header).strip()
        
        data_dict = {}
        for idx, header in enumerate(headers):
            if pd.notna(header) and str(header).strip():
                value = data[idx]
                # Behalte den originalen Datentyp bei
                if pd.notna(value):
                    data_dict[str(header).strip()] = value
        
        df = pd.DataFrame([data_dict])
        
        metadata = {
            'variable_mapping': variable_mapping,
            'source_file': Path(file_path).name,
            'processing_timestamp': datetime.now().isoformat(),
            'sheet_name': pd.ExcelFile(file_path).sheet_names[0]  # Name des ersten Arbeitsblatts
        }
        
        df['_metadata'] = json.dumps(metadata)
        
        return df, metadata
        
    except Exception as e:
        print(f"Fehler beim Verarbeiten von {file_path}: {str(e)}")
        raise

def process_multiple_files(input_dir, output_dir, combine=True):
    """
    Verarbeitet alle Excel-Dateien im Input-Verzeichnis
    combine: Wenn True, werden alle Dateien in ein Parquet geschrieben
    """
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    all_data = []
    all_metadata = []

    for excel_file in input_path.glob('*.xls*'):  # Findet .xls und .xlsx
        print(f"Verarbeite {excel_file.name}")
        
        try:
            # Verarbeite Datei
            df, metadata = process_excel_file(str(excel_file))
            
            if not combine:
                output_file = output_path / f"{excel_file.stem}.parquet"
                df.to_parquet(output_file, index=False)
                print(f"Gespeichert: {output_file}")
                
                metadata_file = output_path / f"{excel_file.stem}_metadata.json"
                with open(metadata_file, 'w', encoding='utf-8') as f:
                    json.dump(metadata, f, ensure_ascii=False, indent=2)
            
            all_data.append(df)
            all_metadata.append(metadata)
            
        except Exception as e:
            print(f"Fehler bei {excel_file.name}: {str(e)}")
            continue
    
    if combine and all_data:
        #Combine
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_file = output_path / "combined_data.parquet"
        combined_df.to_parquet(combined_file, index=False)
        print(f"Gespeichert: {combined_file}")
        
        #Save
        metadata_file = output_path / "all_metadata.json"
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(all_metadata, f, ensure_ascii=False, indent=2)

def quick_check(parquet_file):
    """
    Schnelle Überprüfung einer Parquet-Datei
    """
    df = pd.read_parquet(parquet_file)
    print("\n=== Daten ===")
    display(df.drop(columns=['_metadata']))
    
    print("\n=== Metadata ===")
    metadata = json.loads(df['_metadata'].iloc[0])
    print("\nVariable Mapping:")
    mapping_df = pd.DataFrame.from_dict(metadata['variable_mapping'], 
                                      orient='index', 
                                      columns=['Header'])
    display(mapping_df)
    
    print("\nDatei-Informationen:")
    for key, value in metadata.items():
        if key != 'variable_mapping':
            print(f"{key}: {value}")
    
    return df

if __name__ == "__main__":
    input_directory = "./input"
    output_directory = "./staging"
    
    # Verarbeite alle Dateien
    process_multiple_files(input_directory, output_directory, combine=True)
    
    # Überprüfe das Ergebnis
    quick_check("./staging/combined_data.parquet")

Verarbeite PF_Externes_Reporting_auto_LA203A_BTL_20241112_20241112_20241113_045437.xls
Verarbeite PF_Externes_Reporting_auto_LA203A_BAL_20241112_20241112_20241113_045436.xls
Gespeichert: staging/combined_data.parquet

=== Daten ===


,Berechnungs_Datum,Interne_Fondsnummer,Fondsbezeichnung,ISIN,WKN,Währung,Erfüllte Bedingung,Formel Fee-Berechnung,Formeln nachgelagerte Bedingung,Performance-Prozentsatz,...,Wertzuwachs der Periode,Berechnete Hürde,Verlustvortrag,Ausschüttungen pro Anteil,Hurdle Rate,High Watermark,Performance +/-,berechnete Gebühr,zu buchende Gebühr,Berechnungszyklus
0,12.11.2024,LA203A_BTL,HESPER FUND - GLOBAL SOLUTIONS T-12 CHF,LU1931801754,A2PEEG,CHF,1=1,"((((FPAKTOS*DEVKV(""CHF"";""EUR"")-(SVANB(""HWMAR"")...","(((((FPAKTOS*DEVKV(""CHF"";""EUR"")-(SVANB(""HWMAR""...",0.1,...,0,0,0,0,1,109.94,0,-864.28,0,Täglich
1,12.11.2024,LA203A_BAL,HESPER FUND - GLOBAL SOLUTIONS A-12 EUR,LU1931795501,A2PEEC,EUR,1=1,"(((FPAKTOS-(SVANB(""HWMAR"")-VLVOR)+SAGEB)*100/(...","((((FPAKTOS-(SVANB(""HWMAR"")-VLVOR)+SAGEB)*100/...",0.1,...,0,0,0,0,1,107.70,0,-16189.18,0,Täglich



=== Metadata ===

Variable Mapping:


,Header
KUWE1,Formel Kumul. Wert 1
KUWE2,Formel Kumul. Wert 2
KUWE3,Formel Kumul. Wert 3
PPROZ,Performance-Prozentsatz
VWGPS,Verwaltungsgebühr-Prozentsatz
...,...
Formel REM18,nan
REM19,nan
Formel REM19,nan
REM20,nan



Datei-Informationen:
source_file: PF_Externes_Reporting_auto_LA203A_BTL_20241112_20241112_20241113_045437.xls
processing_timestamp: 2024-11-13T18:44:46.674329
sheet_name: Tabelle1
